In [1]:
from abc import ABC, abstractmethod

# Mediator Interface
class DialogMediator(ABC):
    @abstractmethod
    def notify(self, sender, event):
        pass

# Concrete Mediator
class AuthenticationDialog(DialogMediator):
    def __init__(self):
        self.username_field = UsernameField(self)
        self.password_field = PasswordField(self)
        self.login_button = LoginButton(self)
        self.message_label = MessageLabel(self)

    def notify(self, sender, event):
        if sender == self.login_button and event == "click":
            username = self.username_field.get_text()
            password = self.password_field.get_text()
            if self.authenticate(username, password):
                self.message_label.set_text("Login successful!")
            else:
                self.message_label.set_text("Invalid credentials.")

    def authenticate(self, username, password):
        # Simulate authentication logic
        return username == "admin" and password == "password"

# Colleague Components
class Component(ABC):
    def __init__(self, mediator):
        self.mediator = mediator

class UsernameField(Component):
    def __init__(self, mediator):
        super().__init__(mediator)
        self.text = ""

    def set_text(self, text):
        self.text = text

    def get_text(self):
        return self.text

class PasswordField(Component):
    def __init__(self, mediator):
        super().__init__(mediator)
        self.text = ""

    def set_text(self, text):
        self.text = text

    def get_text(self):
        return self.text

class LoginButton(Component):
    def click(self):
        self.mediator.notify(self, "click")

class MessageLabel(Component):
    def set_text(self, text):
        print(f"Message: {text}")



In [2]:
from abc import ABC, abstractmethod

class ChatMediator(ABC):
    @abstractmethod
    def send_message(self, message: str, sender: 'Participant'):
        pass

    @abstractmethod
    def add_participant(self, participant: 'Participant'):
        pass

class ChatRoom(ChatMediator):
    def __init__(self):
        self.participants: list['Participant'] = []

    def add_participant(self, participant: 'Participant'):
        self.participants.append(participant)
        participant.set_mediator(self)
        print(f"{participant.name} has joined the chat room.")

    def send_message(self, message: str, sender: 'Participant'):
        for participant in self.participants:
            if participant != sender:
                participant.receive(message, sender.name)

class Participant(ABC):
    @abstractmethod
    def set_mediator(self, mediator: ChatMediator):
        pass

    @abstractmethod
    def send(self, message: str):
        pass

    @abstractmethod
    def receive(self, message: str, sender_name: str):
        pass

class UserParticipant(Participant):
    def __init__(self, name: str):
        self.name = name
        self.mediator: ChatMediator = None
        
    def set_mediator(self, mediator: ChatMediator):
        self.mediator = mediator
        
    def send(self, message: str):
        if self.mediator:
            print(f"{self.name} sends: {message}")
            self.mediator.send_message(message, self)
        else:
            print("No mediator to send the message.")

    def receive(self, message: str, sender_name: str):
        print(f"{self.name} received from {sender_name}: {message}")




In [3]:
# Example usage
def main():
    # Create a chat room (mediator)
    chat_room = ChatRoom()

    # Create participants
    alice = UserParticipant("Alice")
    bob = UserParticipant("Bob")
    charlie = UserParticipant("Charlie")

    # Add participants to the chat room
    chat_room.add_participant(alice)
    chat_room.add_participant(bob)
    chat_room.add_participant(charlie)

    print("\n--- Chat Begins ---\n")

    # Participants send messages
    alice.send("Hi everyone!")
    bob.send("Hello Alice!")
    charlie.send("Hey folks, what's up?")

if __name__ == "__main__":
    main()


Alice has joined the chat room.
Bob has joined the chat room.
Charlie has joined the chat room.

--- Chat Begins ---

Alice sends: Hi everyone!
Bob received from Alice: Hi everyone!
Charlie received from Alice: Hi everyone!
Bob sends: Hello Alice!
Alice received from Bob: Hello Alice!
Charlie received from Bob: Hello Alice!
Charlie sends: Hey folks, what's up?
Alice received from Charlie: Hey folks, what's up?
Bob received from Charlie: Hey folks, what's up?


In [4]:
class Mediator:
    def send_message(self, msg, department):
        pass

class Department:
    def __init__(self, mediator=None):
        self.mediator = mediator

    def send_message(self, msg):
        self.mediator.send_message(msg, self)

    def receive_message(self, msg):
        pass

class HRDepartment(Department):
    def receive_message(self, msg):
        print(f"HR Department received: {msg}")

class FinanceDepartment(Department):
    def receive_message(self, msg):
        print(f"Finance Department received: {msg}")

class OfficeMediator(Mediator):
    def __init__(self):
        self.hr = None
        self.finance = None

    def set_hr(self, hr):
        self.hr = hr

    def set_finance(self, finance):
        self.finance = finance

    def send_message(self, msg, department):
        if department == self.hr:
            if self.finance:
                self.finance.receive_message(msg)
        elif department == self.finance:
            if self.hr:
                self.hr.receive_message(msg)

# Test the implementation
mediator = OfficeMediator()
hr = HRDepartment(mediator)
finance = FinanceDepartment(mediator)

mediator.set_hr(hr)
mediator.set_finance(finance)

hr.send_message("HR update for all departments")


Finance Department received: HR update for all departments
